In [1]:
import mysql.connector
import pandas as pd

connection = mysql.connector.connect(
    host= 'localhost',         
    database = 'day37',
    user= 'root',             
    password= '123456'       
)
cursor = connection.cursor()

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sample_docs = [
("Bài luận 1", "Trí tuệ nhân tạo đang phát triển nhanh chóng và tác động lớn đến cuộc sống."),
("Bài luận 2", "Trí tuệ nhân tạo đang phát triển mạnh mẽ và ảnh hưởng nhiều đến đời sống con người."),
("Bài luận 3", "Trí tuệ nhân tạo rất mạnh mẽ và ảnh hưởng nhiều đến đời sống con người."),
("Bài luận 4", "Deep Learning đã mang lại những bước tiến vượt bậc trong lĩnh vực xử lý ngôn ngữ tự nhiên."),
("Bài luận 5", "Robotics và tự động hóa đang thay đổi cách chúng ta làm việc trong các nhà máy."),
("Bài luận 6", "Internet of Things kết nối hàng tỷ thiết bị thông minh trên toàn cầu."),
("Bài luận 7", "Blockchain không chỉ dùng cho tiền điện tử mà còn nhiều ứng dụng khác."),
("Bài luận 8", "Cloud Computing giúp doanh nghiệp tiết kiệm chi phí và tăng hiệu quả."),
("Bài luận 9", "Bảo mật thông tin là thách thức lớn trong kỷ nguyên số."),
("Bài luận 10", "5G sẽ tạo ra cuộc cách mạng trong truyền thông di động.")
]

### Phần 1

### Phần 2

### Phần 3

In [3]:
cursor.execute("""
CREATE TABLE documents (
    doc_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    title VARCHAR(200) NOT NULL,
    content TEXT NOT NULL,
    upload_date TIMESTAMP NOT NULL, 
    last_modified TIMESTAMP NOT NULL
);
""")

In [4]:
cursor.execute("""
CREATE TABLE similarity_scores (
    score_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    doc1_id INTEGER NOT NULL,
    doc2_id INTEGER NOT NULL,
    similarity_score FLOAT NOT NULL, 
    check_date TIMESTAMP NOT NULL,
    FOREIGN KEY (doc1_id) REFERENCES documents(doc_id),
    FOREIGN KEY (doc2_id) REFERENCES documents(doc_id)
);
""")

In [5]:
cursor.execute("""
CREATE TABLE plagiarism_reports (
    report_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    doc_id INTEGER NOT NULL,
    report_date TIMESTAMP NOT NULL,
    status ENUM('pending', 'reviewed', 'confirmed', 'rejected') NOT NULL, 
    reviewer_notes TEXT NOT NULL,
    FOREIGN KEY (doc_id) REFERENCES documents(doc_id)
);
""")

In [6]:
cursor.execute("""
CREATE TABLE document_metadata (
    metadata_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    doc_id INTEGER NOT NULL,
    word_count INT NOT NULL,
    language VARCHAR(50) NOT NULL, 
    category VARCHAR(100) NOT NULL, 
    keywords TEXT NOT NULL,             
    FOREIGN KEY (doc_id) REFERENCES documents(doc_id)
);
""")

### Phần 4